In [ ]:
# Import libraries and dependencies
import os
import pandas as pd
import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation

In [ ]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:
# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes between now and 3 years ago.
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()

# Set the ticker information
tickers = ["AAPL, "]

# Get 3 year's worth of historical price data for Microsoft and Coca-Cola
df_portfolio = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Display sample data
df_portfolio.head()

In [ ]:
#Keep for fun - portfolio api call

str_input= input("Input portfolio tickers here: ")

def input_splice(string):
    """
    docstring goes here
    """
    tickers= str_input.split(", ")
    
    print(tickers)

input_splice(str_input)

for ticker in tickers:
    uri.append(f)